# VoTT to YOLO converter
Notebook with scripts for converting VoTT JSON exported annotations into the format YOLO uses, assumes all annotated images are in the JSON-file generated by VoTT export.

VoTT JSON export format:
- Single file for all annotations.
- In JSON: top, left, width, height.
- In csv: 4 points for defining a rectangle.

YOLO format:
- `.txt` file for each image in the same directory, but with .txt extension.
- Bounding box format: `<object-class>` `<x>` `<y>` `<width>` `<height>`
  - `<x>` - center of the bounding box from the left.
  - `<y>` - center of the bounding box from the top.
  - `<width>` - width of the bounding box.
  - `<height>` - height of the bounding box.
  - `NOTE` : Image dimensions are scaled to be from `0` to `1`.

In [38]:
import json
import csv
import logging
from pathlib import Path
import os

from dotenv import load_dotenv
load_dotenv() 

## Define filepaths

In [39]:
datasetName = 'dragons'
VoTTProjectName = 'dragons'

In [41]:
datasetsRootPath = Path('datasets') / datasetName
datasetsRootPath.mkdir(exist_ok=True, parents=True)

annotationJsonFileVoTT = datasetsRootPath / 'data' / 'obj' / 'vott-json-export' / ( VoTTProjectName + '-export.json')
yoloAnnotationDestinationFolder = datasetsRootPath / 'data' / 'obj'
yoloAnnotationDestinationFolder.mkdir(exist_ok=True, parents=True)

trainingFileListYOLO = datasetsRootPath / 'data' / 'train-obj.txt'
trainingClassDefinitionsYOLO = datasetsRootPath / 'data' / 'obj.names'

logging.basicConfig(filename= datasetsRootPath / 'yolo_conversion.log', level=logging.INFO)

## Define helper methods

In [36]:
def convertVoTTJsonAssetToYOLOFormat(asset:dict, tags: dict, filepath: Path, boundingBoxMinSize: float = 0.0):
    orig_id = asset['asset']['id']
    orig_width = asset['asset']['size']['width'] 
    orig_height = asset['asset']['size']['height']
    filename = asset['asset']['name'].rsplit('.', 1)[0] + '.txt'

    yolo_rows = []
    has_annotations = False

    if asset.get('regions'):
        for region in asset['regions']:
            box_tags = region['tags']
            for tag in box_tags:
                if tag in tags:
                    box = region['boundingBox']
                    box_left = box['left']
                    box_top = box['top']
                    box_height = box['height']
                    box_width = box['width']

                    yolo_object_class = tags[tag]
                    yolo_x = (box_left + box_width/2.0) / orig_width 
                    yolo_y = (box_top + box_height/2.0) / orig_height
                    yolo_width = box_width / orig_width
                    yolo_height = box_height / orig_height

                    if yolo_width < boundingBoxMinSize or yolo_height < boundingBoxMinSize:
                        logging.info(f"Image {asset['asset']['name']} has bounding box smaller than the defined threshold, ignoring")
                        continue

                    has_annotations = True
                    yolo_rows.append([yolo_object_class,yolo_x,yolo_y,yolo_width,yolo_height])

    if not has_annotations:
        logging.info(f"Image {asset['asset']['name']} has no annotations, writing an empty file")
    
    with open(filepath / filename, mode='w') as outfile:
        writer = csv.writer(outfile, delimiter= ' ')
        for row in yolo_rows:
            writer.writerow(row)

def createTrainListFileFromVoTTJsonExport(assets:dict, relativeYoloPath: str, filepath: Path):
    '''
    Params:
        relativeYoloPath (str): Filepath put into the YOLO dataset image list entries
    '''
    open(filepath, 'w').close()
    with open(filepath, mode='a') as outfile:
        for asset in loaded['assets'].values():
            filename = asset['asset']['name']
            writer = csv.writer(outfile, delimiter= ' ')
            writer.writerow([relativeYoloPath+filename])

def createTrainClassesFile(tags:dict, filepath: Path):
    '''Write class names in tags to a file which is readable by YOLO'''
    with open(filepath, mode='w') as outfile:
        writer = csv.writer(outfile, delimiter= ' ')
        for key in tags.keys():
            writer.writerow([key])

## Convert VoTT annotations into the YOLO format and create supplementary files needed by YOLO

In [37]:
with open(annotationJsonFileVoTT) as json_file:
    loaded = json.load(json_file)

# Bounding box classes to use, bounding boxes with other classes are ignored
tags = { 'dragon_face': 0 }
createTrainClassesFile(tags,trainingClassDefinitionsYOLO)

yoloPath = 'data/obj/' # Filepath put into the YOLO dataset image list entries
createTrainListFileFromVoTTJsonExport(loaded['assets'], yoloPath, trainingFileListYOLO)

for asset in loaded['assets'].values():
    convertVoTTJsonAssetToYOLOFormat(asset,tags,yoloAnnotationDestinationFolder,0.0375)

print( str(len(loaded['assets'])) + ' bounding boxes processed' )

21 bounding boxes processed
